In [1]:
import os
from dotenv import load_dotenv


In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY", "your_langchain_api_key_here")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT", "your_langchain_project_here")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY", "your_groq_api_key_here")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.getenv("GROQ_API_KEY", "your_groq_api_key_here")


'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'

In [3]:
from langchain_groq import ChatGroq

model=ChatGroq(model="qwen-qwq-32b")
result=model.invoke("tell me a joke about cats")
print(result)

content='\n<think>\nOkay, the user wants a joke about cats. Let me think of some common cat-related humor. Cats are known for their quirky behaviors, like knocking things off tables, napping in sunny spots, and being a bit aloof. Maybe something with a play on words?\n\nHmm, "Why did the cat sit on the computer?" Oh, right, "To keep an eye on the mouse!" That\'s a classic one. The mouse on the computer refers to the cursor or the actual mouse. It\'s a pun that works because cats chase mice. Let me make sure it\'s not too overused. I think it\'s a solid choice. Alternatively, maybe another angle? Like "Why don\'t cats play poker in the Wild West?" "Too many outlaws!" But that\'s more about the wild west. The first one is better. I\'ll go with the computer and mouse joke. It\'s simple, clean, and directly relates to cats and their instincts. Yep, that should work.\n</think>\n\nSure! Here\'s a light-hearted one:  \n\n**Why did the cat sit on the computer?**  \n*Because it wanted to keep a

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
chain=model| parser
chain.invoke("tell me a joke about cats")

'\n<think>\nOkay, the user wants a joke about cats. Let me think of some common cat-related humor. Cats are known for their independent nature, their love of sitting in boxes, and their occasional mischievousness.\n\nMaybe I can play on the idea that cats always land on their feet. Like a pun with "purr-son" instead of "person." Let me see... \n\nWait, another angle: why did the cat sit on the computer? Because it wanted to keep an eye on the mouse! But "mouse" here refers to the computer mouse, which is a classic pun. That\'s good. It\'s simple and relatable. \n\nAlternatively, there\'s the one about the cat with no tail. "What do you call a cat with no tail? A sit-ter." Hmm, that\'s okay but maybe not as strong. \n\nThe first one with the computer mouse is better. It\'s a common scenario, and the pun is clear. Let me make sure it\'s not too overused. I think it\'s a solid choice. Let me go with that one. \n\nWait, another thought: How do cats dance? They waltz. Because "waltz" sounds

### chaining with some extra stuff using promptTemplate and ChatPromptTemplate


In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert language translator."),
    ("human", "Your task is to convert the following text into '{language}' language:\n\n'{text}'")
])


In [12]:
prompt.format_messages(
    language="hindi",
    text="my name is rehaman , i want to learn agentic AI"
    
)
prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert language translator.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template="Your task is to convert the following text into '{language}' language:\n\n'{text}'"), additional_kwargs={})])

In [17]:
chain = prompt | model

response = chain.invoke({
    "language": "hindi",
    "text": "my name is rehaman, I want to learn agentic AI"
})

print(response)


content='\n<think>\nOkay, I need to translate the user\'s message into Hindi. The original text is: "my name is rehaman, I want to learn agentic AI". Let me break this down.\n\nFirst, "my name is" translates to "मैं का नाम है" in Hindi. \n\n"Rehaman" is a name, so it should remain the same, but in Hindi script it might be written as "रेहमान" to match the phonetics. \n\nThen, "I want to learn" would be "मैं सीखना चाहता हूँ" (assuming the person is male; if gender is unknown, maybe use a neutral form). \n\n"Agentic AI" is a technical term. "Agentic" here likely refers to "agent-based" or "autonomous" AI. In Hindi, "agentic AI" might be translated as "एजेंटिक आई" since it\'s a specific term and not commonly translated. Alternatively, maybe "एजेंट आई" but "agentic" could be a specific term so keeping it in English with Hindi script might be better. \n\nPutting it all together: \n\nमेरा नाम रेहमान है, मैं एजेंटिक आई सीखना चाहता हूँ. \n\nWait, let me check the structure again. \n\n"My name i

In [18]:
chain= prompt|model|parser
response=chain.invoke({
    "language":"german",
    "text":"hii how are you"
})
response

'\n<think>\nOkay, the user wants to translate "hii how are you" into German. Let me start by breaking down the sentence. "Hii" is a casual or informal way of saying "hi" or "hello," maybe a bit more stretched for emphasis. In German, the equivalent would be "Hi" or "Hallo." Since it\'s already "Hi" in English, translating it to German would just be "Hi" as well, but maybe they want it more formal? Wait, the user specified to convert to German, so "Hi" is acceptable.\n\nNext, "how are you?" is the main part. The standard translation for "how are you?" in German is "Wie geht\'s?" which is informal. But if they want a slightly more formal version, it would be "Wie geht es Ihnen?" when addressing someone with "Sie," or "Wie geht es dir?" for informal "du." The original "hii" seems informal, so probably using "dir" is better here. However, the user might just want the standard translation. Let me check for commonality. "Wie geht\'s?" is more colloquial and matches the informal "hii." \n\nPu

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Step 1: Define the JSON parser
parser = JsonOutputParser()

# Step 2: Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an expert translator. Return ONLY this exact JSON format: {{\"translation\": \"...\"}}. No explanations, no markdown, no extra text."
    ),
    (
        "human",
        """Translate the following text into '{language}' language:

Text: {text}

{format_instructions}"""
    )
])

# Step 3: Inject format instructions from the parser
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Step 4: Define the model
model = ChatGroq(model="qwen-qwq-32b")

# Step 5: Create the chain
chain = prompt | model | parser

# Step 6: Invoke the chain
response = chain.invoke({
    "language": "hindi",
    "text": "my name is rehaman, I want to learn agentic AI"
})

print(response)


OutputParserException: Invalid json output: <think>
Okay, the user wants me to translate "my name is rehaman, I want to learn agentic AI" into Hindi. Let me break this down. First, the name "rehaman" – since it's a name, I should keep it as is, maybe in Hindi script? Wait, but the user might prefer it in English. Hmm, the example they gave earlier used the name as is. So probably just transliterate the rest.

"My name is" translates to "मेरा नाम है" in Hindi. Then the name remains "rehaman". Next, "I want to learn" is "मैं सीखना चाहता हूँ" or "मैं सीखना चाहती हूँ" depending on gender. Since the original doesn't specify, maybe use the masculine form as default. 

Then "agentic AI" – "agentic" here likely refers to "agentic AI systems" or "agentic artificial intelligence". The term "agentic" might not have a direct translation. Maybe keep it as is in English or transliterate. Since technical terms are often kept in English, but to make it Hindi, perhaps "एजेंटिक AI" would work. 

Putting

In [29]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an expert translator. Return ONLY this exact JSON format: {{\"translation\": \"...\"}}. No explanations, no markdown, no extra text."
    ),
    (
        "human",
        """Translate the following text into '{language}' language:

Text: {text}

{format_instructions}"""
    )
])

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

chain = prompt | model | parser

response = chain.invoke({
    "language": "hindi",
    "text": "my name is rehaman, I want to learn agentic AI"
})

print(response)


{'translation': 'मेरा नाम रहमान है, मैं एजेंटिक एआई सीखना चाहता हूँ'}
